# SVD between a Model and Satellite data

This notebook shows how to multiply two matrices and calculate SVD. Each matrix is created out a set of GeoTiffs for a series of years. Both matrices should have the same dimension.

For demonstration we will use from a model (spring-index) and from a satellite (AVHRR).

## Dependencies

In [1]:
import java.io.{ByteArrayInputStream, ByteArrayOutputStream, ObjectInputStream, ObjectOutputStream}

import geotrellis.proj4.CRS
import geotrellis.raster.io.geotiff.writer.GeoTiffWriter
import geotrellis.raster.io.geotiff.{SinglebandGeoTiff, _}
import geotrellis.raster.{CellType, DoubleArrayTile, Tile, UByteCellType}
import geotrellis.spark.io.hadoop._
import geotrellis.vector.{Extent, ProjectedExtent}
import org.apache.hadoop.io.SequenceFile.Writer
import org.apache.hadoop.io.{SequenceFile, _}
import org.apache.spark.broadcast.Broadcast
import org.apache.spark.mllib.linalg._
import org.apache.spark.mllib.linalg.distributed._
import org.apache.spark.mllib.stat.{MultivariateStatisticalSummary, Statistics}
import org.apache.spark.rdd.RDD
import org.apache.spark.storage.StorageLevel
import org.apache.spark.{SparkConf, SparkContext}
import spire.syntax.cfor.cfor

import scala.sys.process.Process

## Mode of operation

Here the user can define the mode of operation.
* **rdd_offline_mode**: If false it means the notebook will create all data from scratch and store protected_extent and num_cols_rows into HDFS. Otherwise, these data structures are read from HDFS.

It is also possible to define which directory of GeoTiffs is to be used and on which **band** to run Kmeans. The options are
* **all** which are a multi-band (**8 bands**) GeoTiffs
* Or choose single band ones:
    0. Onset_Greenness_Increase
    1. Onset_Greenness_Maximum
    2. Onset_Greenness_Decrease
    3. Onset_Greenness_Minimum
    4. NBAR_EVI_Onset_Greenness_Minimum
    5. NBAR_EVI_Onset_Greenness_Maximum
    6. NBAR_EVI_Area
    7. Dynamics_QC

<span style="color:red">Note that when using a range **kemans offline mode** is not possible and it will be reset to **online mode**</span>.

### Mode of Operation setup
<a id='mode_of_operation_setup'></a>

In [2]:
var model_rdd_offline_mode = true
var model_matrix_offline_mode = true
var satellite_rdd_offline_mode = true
var satellite_matrix_offline_mode = true

//Using spring-index model
var model_path = "hdfs:///user/hadoop/spring-index/"
var model_dir = "BloomFinalLowPR"
var model_band_num = 0  //First band is 0

//Using AVHRR Satellite data
var satellite_path = "hdfs:///user/hadoop/avhrr/"
var satellite_dir = "SOSTLowPR"
//var satellite_path = "hdfs:///user/hadoop/spring-index/"
//var satellite_dir = "LeafGridmet"
var sat_band_num = 0  //First band is 0

var out_path = "hdfs:///user/emma/svd/spark/" + model_dir + satellite_dir + "3/"
//var out_path = "hdfs:///user/pheno/svd/spark/" + satellite_dir + model_dir + "3/"

//Years between (inclusive) 1989 - 2014
var satellite_first_year = 1989
var satellite_last_year = 2014

//Years between (inclusive) 1980 - 2015
var model_first_year = 1989
var model_last_year = 2014

//Mask
val modToBeMasked = false
val satToBeMasked = false
//val mask_path = "hdfs:///user/hadoop/usa_mask_low.tif"
val mod_mask_path = "hdfs:///user/hadoop/usa_mask_low.tif"
val sat_mask_path = "hdfs:///user/hadoop/usa_mask_low.tif"

//Matrix Mode: 0 Normal, 1 SC, 2 SR
val matrix_mode = 0

val save_rdds = true
val save_matrix = true

model_rdd_offline_mode = true
model_matrix_offline_mode = true
satellite_rdd_offline_mode = true
satellite_matrix_offline_mode = true
model_path = hdfs:///user/hadoop/spring-index/
model_dir = BloomFinalLowPR
model_band_num = 0
satellite_path = hdfs:///user/hadoop/avhrr/
satellite_dir = SOSTLowPR
sat_band_num = 0
out_path = hdfs:///user/pheno/svd/spark/BloomFinalLowPRSOSTLowPR3/
satellite_first_year = 1989
satellite_last_year = 2014
model_first_year = 1989
model_last_year = 2014
modToBeMasked = false
satToBeMasked = false
mod_mask_path = hdfs:///user/hadoop/usa_mask_low.tif
sat_mask_path = hdfs:///user/hadoop/usa_mask_low.tif
matrix_mode = 0


sa...


0


<span style="color:red">DON'T MODIFY ANY PIECE OF CODE FROM HERE ON!!!</span>.


### Mode of operation validation

In [3]:
//Check offline modes
var conf = sc.hadoopConfiguration
var fs = org.apache.hadoop.fs.FileSystem.get(conf)

//Paths to store data structures for Offline runs
var mod_mask_str = ""
var sat_mask_str = ""
if (modToBeMasked)
  mod_mask_str = "_mask"
if (satToBeMasked)
  sat_mask_str = "_mask"

val matrix_mode_str = matrix_mode match {
  case 1 => "_Sc_Mc"
  case 2 => "_Sr_Mr"
  case _ => ""
}

var model_grid0_path = out_path + model_dir + "_grid0"
var model_grid0_index_path = out_path + model_dir + "_grid0_index"
var satellite_grid0_path = out_path + satellite_dir + "_grid0"
var satellite_grid0_index_path = out_path + satellite_dir + "_grid0_index"

var model_grid_path = out_path + model_dir + "_grid"
var satellite_grid_path = out_path + satellite_dir + "_grid"
var model_matrix_path = out_path + model_dir + "_matrix"
var satellite_matrix_path = out_path + satellite_dir + "_matrix"
var metadata_path = out_path + model_dir + "_metadata"

Waiting for a Spark session to start...

conf = Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml, file:/usr/lib/spark-2.1.1-bin-without-hadoop/conf/hive-site.xml
fs = DFS[DFSClient[clientName=DFSClient_NONMAPREDUCE_1091801106_42, ugi=emma (auth:SIMPLE)]]
model_grid0_path = hdfs:///user/pheno/svd/spark/BloomFinalLowPRSOSTLowPR3/BloomFinalLowPR_grid0
model_grid0_index_path = hdfs:///user/pheno/svd/spark/BloomFinalLowPRSOSTLowPR3/BloomFinalLowPR_grid0_index
satellite_grid0_path = hdfs:///user/pheno/svd/spark/BloomFinalLowPRSOSTLowPR3/SOSTLowPR_grid0


mod_mask_str: String = ""
sat_mask_str: String = ""
matrix_mode_str: String = ""
satellit...


hdfs:///user/pheno/svd/spark/BloomFinalLowPRSOSTLowPR3/SOSTLowPR_grid0

In [4]:
var sc_path = out_path + model_dir + "_sc"
var mc_path = out_path + model_dir + "_mc"

val model_rdd_offline_exists = fs.exists(new org.apache.hadoop.fs.Path(model_grid_path))
val model_matrix_offline_exists = fs.exists(new org.apache.hadoop.fs.Path(model_matrix_path))
val satellite_rdd_offline_exists = fs.exists(new org.apache.hadoop.fs.Path(satellite_grid_path))
val satellite_matrix_offline_exists = fs.exists(new org.apache.hadoop.fs.Path(satellite_matrix_path))

if (model_rdd_offline_mode != model_rdd_offline_exists) {
  println("\"Load GeoTiffs\" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to " + model_rdd_offline_exists.toString())
  model_rdd_offline_mode = model_rdd_offline_exists
}

if (model_matrix_offline_mode != model_matrix_offline_exists) {
  println("\"Matrix\" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to " + model_matrix_offline_exists.toString())
  model_matrix_offline_mode = model_matrix_offline_exists
}

var model_skip_rdd = false
if (model_matrix_offline_exists) {
    println("Since we have a matrix, the load of the grids RDD will be skipped!!!")
    model_skip_rdd = true
}

if (satellite_rdd_offline_mode != satellite_rdd_offline_exists) {
  println("\"Load GeoTiffs\" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to " + satellite_rdd_offline_exists.toString())
  satellite_rdd_offline_mode = satellite_rdd_offline_exists
}

if (satellite_matrix_offline_mode != satellite_matrix_offline_exists) {
  println("\"Matrix\" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to " + satellite_matrix_offline_exists.toString())
  satellite_matrix_offline_mode = satellite_matrix_offline_exists
}

var satellite_skip_rdd = false
if (satellite_matrix_offline_exists) {
    println("Since we have a matrix, the load of the grids RDD will be skipped!!!")
    satellite_skip_rdd = true
}

//Years
val model_years = 1989 to 2014
val satellite_years = 1989 to 2014

if (!satellite_years.contains(satellite_first_year) || !(satellite_years.contains(satellite_last_year))) {
  println("Invalid range of years for " + satellite_dir + ". I should be between " + satellite_first_year + " and " + satellite_last_year)
  System.exit(0)
}

if (!model_years.contains(model_first_year) || !(model_years.contains(model_last_year))) {
  println("Invalid range of years for " + model_dir + ". I should be between " + model_first_year + " and " + model_last_year)
  System.exit(0)
}

if ( ((satellite_last_year - model_first_year) > (model_last_year - model_first_year)) || ((satellite_last_year - model_first_year) > (model_last_year - model_first_year))) {
  println("The range of years for each data set should be of the same length.");
  System.exit(0)
}

var model_years_range = (model_years.indexOf(model_first_year), model_years.indexOf(model_last_year))
var satellite_years_range = (satellite_years.indexOf(satellite_first_year), satellite_years.indexOf(satellite_last_year))

//Global variables
var projected_extent = new ProjectedExtent(new Extent(0,0,0,0), CRS.fromName("EPSG:3857"))
var model_grid0: RDD[(Long, Double)] = sc.emptyRDD
var model_grid0_index: RDD[Long] = sc.emptyRDD
var mod_grids_RDD: RDD[Array[Double]] = sc.emptyRDD
var model_grids_RDD: RDD[Array[Double]] = sc.emptyRDD
var model_grids: RDD[Array[Double]] = sc.emptyRDD
val rows = sc.parallelize(Array[Double]()).map(m => Vectors.dense(m))
var model_summary: MultivariateStatisticalSummary = new RowMatrix(rows).computeColumnSummaryStatistics()
var model_std :Array[Double] = new Array[Double](0)

var satellite_grid0: RDD[(Long, Double)] = sc.emptyRDD
var satellite_grid0_index: RDD[Long] = sc.emptyRDD
var sat_grids_RDD: RDD[Array[Double]] = sc.emptyRDD
var satellite_grids_RDD: RDD[Array[Double]] = sc.emptyRDD
var satellite_grids: RDD[Array[Double]] = sc.emptyRDD
var satellite_summary: MultivariateStatisticalSummary = new RowMatrix(rows).computeColumnSummaryStatistics()
var satellite_std :Array[Double] = new Array[Double](0)

var num_cols_rows :(Int, Int) = (0, 0)
var cellT :CellType = UByteCellType
var mod_mask_tile0 :Tile = new SinglebandGeoTiff(geotrellis.raster.ArrayTile.empty(cellT, num_cols_rows._1, num_cols_rows._2), projected_extent.extent, projected_extent.crs, Tags.empty, GeoTiffOptions.DEFAULT).tile
var sat_mask_tile0 :Tile = new SinglebandGeoTiff(geotrellis.raster.ArrayTile.empty(cellT, num_cols_rows._1, num_cols_rows._2), projected_extent.extent, projected_extent.crs, Tags.empty, GeoTiffOptions.DEFAULT).tile
var satellite_cells_size :Long = 0
var model_cells_size :Long = 0
var t0 : Long = 0
var t1 : Long = 0
print(t0)

"Load GeoTiffs" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to false
"Matrix" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to false
"Load GeoTiffs" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to false
"Matrix" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to false
[Stage 2:=====================================================>   (28 + 2) / 30]0

sc_path = hdfs:///user/pheno/svd/spark/BloomFinalLowPRSOSTLowPR3/BloomFinalLowPR_sc
mc_path = hdfs:///user/pheno/svd/spark/BloomFinalLowPRSOSTLowPR3/BloomFinalLowPR_mc
model_rdd_offline_exists = false
model_matrix_offline_exists = false
satellite_rdd_offline_exists = false
satellite_matrix_offline_exists = false
model_skip_rdd = false
satellite_skip_rdd = false
model_years = Range(1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014)
satellite_years = Range(1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 200...


Range(1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014)

## Functions to (de)serialize any structure into Array[Byte]

In [5]:
def serialize(value: Any): Array[Byte] = {
    val out_stream: ByteArrayOutputStream = new ByteArrayOutputStream()
    val obj_out_stream = new ObjectOutputStream(out_stream)
    obj_out_stream.writeObject(value)
    obj_out_stream.close
    out_stream.toByteArray
}

def deserialize(bytes: Array[Byte]): Any = {
    val obj_in_stream = new ObjectInputStream(new ByteArrayInputStream(bytes))
    val value = obj_in_stream.readObject
    obj_in_stream.close
    value
}

serialize: (value: Any)Array[Byte]
deserialize: (bytes: Array[Byte])Any


## Load GeoTiffs

Using GeoTrellis all GeoTiffs of a directory will be loaded into a RDD. Using the RDD, we extract a grid from the first file to lated store the Kmeans cluster_IDS, we build an Index for populate such grid and we filter out here all NaN values.

In [6]:
t0 = System.nanoTime()

//Load Mask
if (modToBeMasked) {
  val mask_tiles_RDD = sc.hadoopGeoTiffRDD(mod_mask_path).values
  val mask_tiles_withIndex = mask_tiles_RDD.zipWithIndex().map { case (e, v) => (v, e) }
  mod_mask_tile0 = (mask_tiles_withIndex.filter(m => m._1 == 0).filter(m => !m._1.isNaN).values.collect()) (0)
}

if (satToBeMasked) {
  val mask_tiles_RDD = sc.hadoopGeoTiffRDD(sat_mask_path).values
  val mask_tiles_withIndex = mask_tiles_RDD.zipWithIndex().map { case (e, v) => (v, e) }
  sat_mask_tile0 = (mask_tiles_withIndex.filter(m => m._1 == 0).filter(m => !m._1.isNaN).values.collect()) (0)
}

//Local variables
val pattern: String = "tif"
val satellite_filepath: String = satellite_path + satellite_dir
val model_filepath: String = model_path + "/" + model_dir

t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

Elapsed time: 92488ns


t0 = 14262587364774
pattern = tif
satellite_filepath = hdfs:///user/hadoop/avhrr/SOSTLowPR
model_filepath = hdfs:///user/hadoop/spring-index//BloomFinalLowPR
t1 = 14262587457262


14262587457262

### Satellite data

In [7]:
t0 = System.nanoTime()
if (satellite_rdd_offline_mode) {
  satellite_grids_RDD = sc.objectFile(satellite_grid_path)
  satellite_grid0 = sc.objectFile(satellite_grid0_path)
  satellite_grid0_index = sc.objectFile(satellite_grid0_index_path)
} else {
  //Lets load MODIS Singleband GeoTiffs and return RDD just with the tiles.
  if (sat_band_num == 0) {
    val satellite_geos_RDD = sc.hadoopGeoTiffRDD(satellite_filepath, pattern)
    val satellite_tiles_RDD = satellite_geos_RDD.values

    if (satToBeMasked) {
      val mask_tile_broad: Broadcast[Tile] = sc.broadcast(sat_mask_tile0)
      sat_grids_RDD = satellite_tiles_RDD.map(m => m.localInverseMask(mask_tile_broad.value, 1, -2000).toArrayDouble().map(m => if (m == -1000) Double.NaN else m))
    } else {
      sat_grids_RDD = satellite_tiles_RDD.map(m => m.toArrayDouble())
    }
  } else {
    val satellite_geos_RDD = sc.hadoopMultibandGeoTiffRDD(satellite_filepath, pattern)
    val satellite_tiles_RDD = satellite_geos_RDD.values

    val band_numB: Broadcast[Int] = sc.broadcast(sat_band_num)
    if (satToBeMasked) {
      val mask_tile_broad: Broadcast[Tile] = sc.broadcast(sat_mask_tile0)
      sat_grids_RDD = satellite_tiles_RDD.map(m => m.band(band_numB.value).localInverseMask(mask_tile_broad.value, 1, -2000).toArrayDouble().map(m => if (m == -1000) Double.NaN else m))
    } else {
      sat_grids_RDD = satellite_tiles_RDD.map(m => m.band(band_numB.value).toArrayDouble())
    }
  }

  //Get Index for each Cell
  val grids_withIndex = sat_grids_RDD.zipWithIndex().map { case (e, v) => (v, e) }
  if (satToBeMasked) {
    satellite_grid0_index = grids_withIndex.filter(m => m._1 == 0).values.flatMap(m => m).zipWithIndex.filter(m => m._1 != -2000.0).map { case (v, i) => (i) }
  } else {
    satellite_grid0_index = grids_withIndex.filter(m => m._1 == 0).values.flatMap(m => m).zipWithIndex.map { case (v, i) => (i) }
  }

  //Get the Tile's grid
  satellite_grid0 = grids_withIndex.filter(m => m._1 == 0).values.flatMap(m => m).zipWithIndex.map { case (v, i) => (i, v) }

  //Lets filter out NaN
  if (satToBeMasked) {
    satellite_grids_RDD = sat_grids_RDD.map(m => m.filter(m => m != -2000.0))
  } else {
    satellite_grids_RDD = sat_grids_RDD
  }

  //Store in HDFS
  if (save_rdds) {
    satellite_grids_RDD.saveAsObjectFile(satellite_grid_path)
    satellite_grid0.saveAsObjectFile(satellite_grid0_path)
    satellite_grid0_index.saveAsObjectFile(satellite_grid0_index_path)
  }
}
val satellite_grids_withIndex = satellite_grids_RDD.zipWithIndex().map { case (e, v) => (v, e) }

//Filter out the range of years:
satellite_grids = satellite_grids_withIndex.filterByRange(satellite_years_range._1, satellite_years_range._2).values
satellite_grids.persist(StorageLevel.DISK_ONLY)

//Collect Stats:
satellite_summary = Statistics.colStats(satellite_grids.map(m => Vectors.dense(m)))
satellite_std = satellite_summary.variance.toArray.map(m => scala.math.sqrt(m))

satellite_cells_size = satellite_grid0_index.count().toInt
println("Number of cells is: " + satellite_cells_size)
t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

[Stage 12:===========================================>              (3 + 1) / 4]Number of cells is: 30870
Elapsed time: 17739767427ns


14282777110158

t0 = 14265037342731
satellite_grids_withIndex = MapPartitionsRDD[43] at map at <console>:146
satellite_grids = MapPartitionsRDD[45] at values at <console>:149
satellite_summary = org.apache.spark.mllib.stat.MultivariateOnlineSummarizer@4c32315
satellite_std = [D@7d6a74e1
satellite_cells_size = 30870
t1 = 14282777110158


### Model data

In [8]:
t0 = System.nanoTime()

if (model_rdd_offline_mode) {
  model_grids_RDD = sc.objectFile(model_grid_path)
  model_grid0 = sc.objectFile(model_grid0_path)
  model_grid0_index = sc.objectFile(model_grid0_index_path)
  val metadata = sc.sequenceFile(metadata_path, classOf[IntWritable], classOf[BytesWritable]).map(_._2.copyBytes()).collect()
  projected_extent = deserialize(metadata(0)).asInstanceOf[ProjectedExtent]
  num_cols_rows = (deserialize(metadata(1)).asInstanceOf[Int], deserialize(metadata(2)).asInstanceOf[Int])
  cellT = deserialize(metadata(3)).asInstanceOf[CellType]
} else {
  if (model_band_num == 0) {
    val model_geos_RDD = sc.hadoopGeoTiffRDD(model_filepath, pattern)
    val model_tiles_RDD = model_geos_RDD.values

    //Retrieve the number of cols and rows of the Tile's grid
    val tiles_withIndex = model_tiles_RDD.zipWithIndex().map { case (v, i) => (i, v) }
    val tile0 = (tiles_withIndex.filter(m => m._1 == 0).values.collect()) (0)

    num_cols_rows = (tile0.cols, tile0.rows)
    cellT = tile0.cellType

    //Retrieve the ProjectExtent which contains metadata such as CRS and bounding box
    val projected_extents_withIndex = model_geos_RDD.keys.zipWithIndex().map { case (e, v) => (v, e) }
    projected_extent = (projected_extents_withIndex.filter(m => m._1 == 0).values.collect()) (0)

    if (modToBeMasked) {
      val mask_tile_broad: Broadcast[Tile] = sc.broadcast(mod_mask_tile0)
      mod_grids_RDD = model_tiles_RDD.map(m => m.localInverseMask(mask_tile_broad.value, 1, -1000).toArrayDouble())
    } else {
      mod_grids_RDD = model_tiles_RDD.map(m => m.toArrayDouble())
    }
  } else {
    val model_geos_RDD = sc.hadoopMultibandGeoTiffRDD(model_filepath, pattern)
    val model_tiles_RDD = model_geos_RDD.values

    //Retrieve the number of cols and rows of the Tile's grid
    val tiles_withIndex = model_tiles_RDD.zipWithIndex().map { case (v, i) => (i, v) }
    val tile0 = (tiles_withIndex.filter(m => m._1 == 0).values.collect()) (0)

    num_cols_rows = (tile0.cols, tile0.rows)
    cellT = tile0.cellType

    //Retrieve the ProjectExtent which contains metadata such as CRS and bounding box
    val projected_extents_withIndex = model_geos_RDD.keys.zipWithIndex().map { case (e, v) => (v, e) }
    projected_extent = (projected_extents_withIndex.filter(m => m._1 == 0).values.collect()) (0)

    val band_numB: Broadcast[Int] = sc.broadcast(model_band_num)
    if (modToBeMasked) {
      val mask_tile_broad: Broadcast[Tile] = sc.broadcast(mod_mask_tile0)
      mod_grids_RDD = model_tiles_RDD.map(m => m.band(band_numB.value).localInverseMask(mask_tile_broad.value, 1, -1000).toArrayDouble())
    } else {
      mod_grids_RDD = model_tiles_RDD.map(m => m.band(band_numB.value).toArrayDouble())
    }
  }

  //Get Index for each Cell
  val grids_withIndex = mod_grids_RDD.zipWithIndex().map { case (e, v) => (v, e) }
  if (modToBeMasked) {
    model_grid0_index = grids_withIndex.filter(m => m._1 == 0).values.flatMap(m => m).zipWithIndex.filter(m => m._1 != -1000.0).map { case (v, i) => (i) }
  } else {
    model_grid0_index = grids_withIndex.filter(m => m._1 == 0).values.flatMap(m => m).zipWithIndex.map { case (v, i) => (i) }
  }

  //Get the Tile's grid
  model_grid0 = grids_withIndex.filter(m => m._1 == 0).values.flatMap( m => m).zipWithIndex.map{case (v,i) => (i,v)}

  //Lets filter out NaN
  if (modToBeMasked) {
    model_grids_RDD = mod_grids_RDD.map(m => m.filter(m => m != -1000.0))
  } else {
    model_grids_RDD = mod_grids_RDD
  }

  //Store data in HDFS
  model_grids_RDD.saveAsObjectFile(model_grid_path)
  model_grid0.saveAsObjectFile(model_grid0_path)
  model_grid0_index.saveAsObjectFile(model_grid0_index_path)

  val writer: SequenceFile.Writer = SequenceFile.createWriter(conf,
    Writer.file(metadata_path),
    Writer.keyClass(classOf[IntWritable]),
    Writer.valueClass(classOf[BytesWritable])
  )

  writer.append(new IntWritable(1), new BytesWritable(serialize(projected_extent)))
  writer.append(new IntWritable(2), new BytesWritable(serialize(num_cols_rows._1)))
  writer.append(new IntWritable(3), new BytesWritable(serialize(num_cols_rows._2)))
  writer.append(new IntWritable(4), new BytesWritable(serialize(cellT)))
  writer.hflush()
  writer.close()
}

val model_grids_withIndex = model_grids_RDD.zipWithIndex().map { case (e, v) => (v, e) }

//Filter out the range of years:
model_grids = model_grids_withIndex.filterByRange(model_years_range._1, model_years_range._2).values
//model_grids.persist(StorageLevel.DISK_ONLY)

//Collect Stats:
model_summary = Statistics.colStats(model_grids.map(m => Vectors.dense(m)))
//model_std = model_summary.variance.toArray.map(m => scala.math.sqrt(m))

var model_tile0_index: RDD[Double] = model_grids_withIndex.filter(m => m._1 == 0).values.flatMap(m => m)
model_cells_size = model_tile0_index.count().toInt

t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

Elapsed time: 13281679586ns                                                     


t0 = 14286059168480
model_grids_withIndex = MapPartitionsRDD[83] at map at <console>:187
model_grids = MapPartitionsRDD[85] at values at <console>:190
model_summary = org.apache.spark.mllib.stat.MultivariateOnlineSummarizer@c81cdf7
model_tile0_index = MapPartitionsRDD[93] at flatMap at <console>:197
model_cells_size = 30870
t1 = 14299340848066


14299340848066

## Matrices

### Satellite

In [9]:
//Satellite
val satellite_cells_sizeB = sc.broadcast(satellite_cells_size)
val satellite_mat: RowMatrix = new RowMatrix(satellite_grids.map(m => m.zipWithIndex).map(m => m.filter(!_._1.isNaN)).map(m => Vectors.sparse(satellite_cells_sizeB.value.toInt, m.map(v => v._2), m.map(v => v._1))))

// Split the matrix into one number per line.
val sat_byColumnAndRow = satellite_mat.rows.zipWithIndex.map {
  case (row, rowIndex) => row.toArray.zipWithIndex.map {
    case (number, columnIndex) => new MatrixEntry(rowIndex, columnIndex, number)
  }
}.flatMap(x => x)
val satellite_blockMatrix: BlockMatrix = new CoordinateMatrix(sat_byColumnAndRow).toBlockMatrix()
//val satellite_blockMatrix: BlockMatrix = new CoordinateMatrix(sat_byColumnAndRow).transpose().toBlockMatrix()

[Stage 28:=====================================================>  (24 + 1) / 25]

satellite_cells_sizeB = Broadcast(30)
satellite_mat = org.apache.spark.mllib.linalg.distributed.RowMatrix@5256b241
sat_byColumnAndRow = MapPartitionsRDD[99] at flatMap at <console>:67
satellite_blockMatrix = org.apache.spark.mllib.linalg.distributed.BlockMatrix@328d26ec


org.apache.spark.mllib.linalg.distributed.BlockMatrix@328d26ec

### SC

In [10]:
//SC
val sc_exists = fs.exists(new org.apache.hadoop.fs.Path(sc_path))
var Sc :BlockMatrix = null
if (sc_exists) {
  val rdd_indexed_rows :RDD[IndexedRow]= sc.objectFile(sc_path)
  Sc = new IndexedRowMatrix(rdd_indexed_rows).toBlockMatrix()
} else {
  val satellite_M_1_Gc = sc.parallelize(Array[Vector](satellite_summary.mean)).map(m => Vectors.dense(m.toArray))
  val satellite_M_1_Gc_RowM: RowMatrix = new RowMatrix(satellite_M_1_Gc)
  val sat_M_1_Gc_byColumnAndRow = satellite_M_1_Gc_RowM.rows.zipWithIndex.map {
    case (row, rowIndex) => row.toArray.zipWithIndex.map {
      case (number, columnIndex) => new MatrixEntry(rowIndex, columnIndex, number)
    }
  }.flatMap(x => x)
  val satellite_M_1_Gc_blockMatrix = new CoordinateMatrix(sat_M_1_Gc_byColumnAndRow).toBlockMatrix()

  val sat_matrix_Nt_1 = new Array[Double](satellite_grids.count().toInt)
  //satellite_grids.unpersist(false)
  for (i <- 0 until sat_matrix_Nt_1.length)
    sat_matrix_Nt_1(i) = 1
  val satellite_M_Nt_1 = sc.parallelize(sat_matrix_Nt_1).map(m => Vectors.dense(m))
  val satellite_M_Nt_1_RowM: RowMatrix = new RowMatrix(satellite_M_Nt_1)
  val sat_M_Nt_1_byColumnAndRow = satellite_M_Nt_1_RowM.rows.zipWithIndex.map {
    case (row, rowIndex) => row.toArray.zipWithIndex.map {
      case (number, columnIndex) => new MatrixEntry(rowIndex, columnIndex, number)
    }
  }.flatMap(x => x)
  val satellite_M_Nt_1_blockMatrix = new CoordinateMatrix(sat_M_Nt_1_byColumnAndRow).toBlockMatrix()
  val satellite_M_Nt_Gc_blockMatrix = satellite_M_Nt_1_blockMatrix.multiply(satellite_M_1_Gc_blockMatrix)

  //Sc = satellite_blockMatrix.subtract(satellite_M_Nt_Gc_blockMatrix)
  val joined_mat :RDD[ (Long, (Array[Double], Array[Double]))] = satellite_blockMatrix.toIndexedRowMatrix().rows.map(m => (m.index, m.vector.toArray)).join(satellite_M_Nt_Gc_blockMatrix.toCoordinateMatrix().toIndexedRowMatrix().rows.map(m => (m.index, m.vector.toArray)))
  Sc = (new CoordinateMatrix(joined_mat.map {case (row_index, (a,b)) => a.zip(b).map(m => m._1-m._2).zipWithIndex.map{ case (v,col_index) => new MatrixEntry(row_index, col_index,v)}}.flatMap(m => m))).toBlockMatrix() 

  //save to disk
  Sc.toIndexedRowMatrix().rows.saveAsObjectFile(sc_path)
}

[Stage 62:======================================================> (29 + 1) / 30]

sc_exists = false
Sc = org.apache.spark.mllib.linalg.distributed.BlockMatrix@44b7f109


org.apache.spark.mllib.linalg.distributed.BlockMatrix@44b7f109

### Model Data

In [11]:
//Model
val model_cells_sizeB = sc.broadcast(model_cells_size)
val model_mat: RowMatrix = new RowMatrix(model_grids.map(m => m.zipWithIndex).map(m => m.filter(!_._1.isNaN)).map(m => Vectors.sparse(model_cells_sizeB.value.toInt, m.map(v => v._2), m.map(v => v._1))))

// Split the matrix into one number per line.
val mod_byColumnAndRow = model_mat.rows.zipWithIndex.map {
  case (row, rowIndex) => row.toArray.zipWithIndex.map {
    case (number, columnIndex) => new MatrixEntry(rowIndex, columnIndex, number)
  }
}.flatMap(x => x)
val model_blockMatrix: BlockMatrix = new CoordinateMatrix(mod_byColumnAndRow).transpose().toBlockMatrix()
//val model_blockMatrix: BlockMatrix = new CoordinateMatrix(mod_byColumnAndRow).toBlockMatrix()

[Stage 64:====================================>                   (17 + 4) / 26]

model_cells_sizeB = Broadcast(54)
model_mat = org.apache.spark.mllib.linalg.distributed.RowMatrix@7ce409a4
mod_byColumnAndRow = MapPartitionsRDD[160] at flatMap at <console>:67
model_blockMatrix = org.apache.spark.mllib.linalg.distributed.BlockMatrix@64b9855c


org.apache.spark.mllib.linalg.distributed.BlockMatrix@64b9855c

### Mc

In [12]:
//MC
val mc_exists = fs.exists(new org.apache.hadoop.fs.Path(mc_path))
var Mc :BlockMatrix = null
if (mc_exists) {
  val rdd_indexed_rows :RDD[IndexedRow]= sc.objectFile(mc_path)
  Mc = new IndexedRowMatrix(rdd_indexed_rows).toBlockMatrix()
} else {
  val model_M_1_Gc = sc.parallelize(Array[Vector](model_summary.mean)).map(m => Vectors.dense(m.toArray))
  val model_M_1_Gc_RowM: RowMatrix = new RowMatrix(model_M_1_Gc)
  val mod_M_1_Gc_byColumnAndRow = model_M_1_Gc_RowM.rows.zipWithIndex.map {
    case (row, rowIndex) => row.toArray.zipWithIndex.map {
      case (number, columnIndex) => new MatrixEntry(rowIndex, columnIndex, number)
    }
  }.flatMap(x => x)
  val model_M_1_Gc_blockMatrix = new CoordinateMatrix(mod_M_1_Gc_byColumnAndRow).toBlockMatrix()

  val model_matrix_Nt_1 = new Array[Double](model_grids.count().toInt)
  //model_grids.unpersist(false)

  for (i <- 0 until model_matrix_Nt_1.length)
    model_matrix_Nt_1(i) = 1
  val model_M_Nt_1 = sc.parallelize(model_matrix_Nt_1).map(m => Vectors.dense(m))
  val model_M_Nt_1_RowM: RowMatrix = new RowMatrix(model_M_Nt_1)
  val mod_M_Nt_1_byColumnAndRow = model_M_Nt_1_RowM.rows.zipWithIndex.map {
    case (row, rowIndex) => row.toArray.zipWithIndex.map {
      case (number, columnIndex) => new MatrixEntry(rowIndex, columnIndex, number)
    }
  }.flatMap(x => x)
  val model_M_Nt_1_blockMatrix = new CoordinateMatrix(mod_M_Nt_1_byColumnAndRow).toBlockMatrix()
  val model_M_Nt_Gc_blockMatrix = model_M_Nt_1_blockMatrix.multiply(model_M_1_Gc_blockMatrix)
  val model_M_Gc_Nt_blockMatrix = model_M_Nt_Gc_blockMatrix.transpose
  
  //Mc = model_blockMatrix.subtract(model_M_Gc_Nt_blockMatrix)
  val joined_mat :RDD[ (Long, (Array[Double], Array[Double]))] = model_blockMatrix.toIndexedRowMatrix().rows.map( m => (m.index, m.vector.toArray)).join(model_M_Gc_Nt_blockMatrix.toIndexedRowMatrix().rows.map(m => (m.index, m.vector.toArray)))
  Mc = (new CoordinateMatrix(joined_mat.map {case (row_index, (a,b)) => a.zip(b).map(m => m._1-m._2).zipWithIndex.map{ case (v,col_index) => new MatrixEntry(row_index, col_index,v)}}.flatMap(m => m))).toBlockMatrix()

  //save to disk
  Mc.toIndexedRowMatrix().rows.saveAsObjectFile(mc_path)
}

[Stage 97:====================================================>   (28 + 2) / 30]

mc_exists = false
Mc = org.apache.spark.mllib.linalg.distributed.BlockMatrix@58df42d2


org.apache.spark.mllib.linalg.distributed.BlockMatrix@58df42d2

### Sr

In [13]:
//SR
var Sr :BlockMatrix = null

Sr: org.apache.spark.mllib.linalg.distributed.BlockMatrix = null


### Mr

In [14]:
//MR
var Mr :BlockMatrix = null

Mr: org.apache.spark.mllib.linalg.distributed.BlockMatrix = null


## Matrix Multiplication

In [15]:
//Matrix Multiplication
var matrix_mul :IndexedRowMatrix = null
var n_components :Long = 0

//Normal Matrix
if (matrix_mode == 0) {
  model_blockMatrix.persist(StorageLevel.DISK_ONLY)
  satellite_blockMatrix.persist(StorageLevel.DISK_ONLY)
  matrix_mul = model_blockMatrix.toIndexedRowMatrix().multiply(satellite_blockMatrix.toLocalMatrix())
  //matrix_mul = satellite_blockMatrix.toIndexedRowMatrix().multiply(model_blockMatrix.toLocalMatrix())
  n_components = List(model_blockMatrix.numRows(), model_blockMatrix.numCols(), satellite_blockMatrix.numRows(), satellite_blockMatrix.numCols()).min
}

//SC Matrix
if (matrix_mode == 1) {
  Mc.persist(StorageLevel.DISK_ONLY)
  Sc.persist(StorageLevel.DISK_ONLY)
  matrix_mul = Mc.toIndexedRowMatrix().multiply(Sc.toLocalMatrix())
  n_components = List(Mc.numRows(), Mc.numCols(), Sc.numRows(), Sc.numCols()).min
}

//SR Matrix
if (matrix_mode == 2) {
  Mr.persist(StorageLevel.DISK_ONLY)
  Sr.persist(StorageLevel.DISK_ONLY)
  matrix_mul = Mr.toIndexedRowMatrix().multiply(Sr.toLocalMatrix())
  n_components = List(Mr.numRows(), Mr.numCols(), Sr.numRows(), Sr.numCols()).min
}

n_components = 3

[Stage 101:===============================================>         (5 + 1) / 6]

matrix_mul = org.apache.spark.mllib.linalg.distributed.IndexedRowMatrix@7049e690
n_components = 3
n_components = 3


3

## Paths to save GeoTiffs

In [17]:
//Paths to save GeoTiffs
var u_geotiff_hdfs_paths :Array[String] = Array.fill[String](n_components.toInt)("")
var u_geotiff_tmp_paths :Array[String] = Array.fill[String](n_components.toInt)("")
var v_geotiff_hdfs_paths :Array[String] = Array.fill[String](n_components.toInt)("")
var v_geotiff_tmp_paths :Array[String] = Array.fill[String](n_components.toInt)("")

//Create dirs in HDFS
var cmd = "hadoop dfs -mkdir " + out_path + "u_tiffs/"
Process(cmd)!

cmd = "hadoop dfs -mkdir " + out_path + "v_tiffs/"
Process(cmd)!

cfor(0)(_ < n_components, _ + 1) { k =>
  u_geotiff_hdfs_paths(k) =  out_path + "u_tiffs/svd_u_" + k + "_" + n_components + matrix_mode_str + ".tif"
  u_geotiff_tmp_paths(k) = "/tmp/svd_u_" + k + "_" + n_components + matrix_mode_str + ".tif"
  if (fs.exists(new org.apache.hadoop.fs.Path(u_geotiff_hdfs_paths(k)))) {
    println("There is already a GeoTiff with the path: " + u_geotiff_hdfs_paths(k) + ". Please make either a copy or move it to another location, otherwise, it will be over-written.")
  }
    
  v_geotiff_hdfs_paths(k) =  out_path + "v_tiffs/svd_v_" + k + "_" + n_components + matrix_mode_str + ".tif"
  v_geotiff_tmp_paths(k) = "/tmp/svd_v_" + k + "_" + n_components + matrix_mode_str + ".tif"
  if (fs.exists(new org.apache.hadoop.fs.Path(v_geotiff_hdfs_paths(k)))) {
    println("There is already a GeoTiff with the path: " + v_geotiff_hdfs_paths(k) + ". Please make either a copy or move it to another location, otherwise, it will be over-written.")
  }
}

DEPRECATED: Use of this script to execute hdfs command is deprecated.
Instead use the hdfs command for it.

DEPRECATED: Use of this script to execute hdfs command is deprecated.
Instead use the hdfs command for it.



u_geotiff_hdfs_paths = Array(hdfs:///user/pheno/svd/spark/BloomFinalLowPRSOSTLowPR3/u_tiffs/svd_u_0_3.tif, hdfs:///user/pheno/svd/spark/BloomFinalLowPRSOSTLowPR3/u_tiffs/svd_u_1_3.tif, hdfs:///user/pheno/svd/spark/BloomFinalLowPRSOSTLowPR3/u_tiffs/svd_u_2_3.tif)
u_geotiff_tmp_paths = Array(/tmp/svd_u_0_3.tif, /tmp/svd_u_1_3.tif, /tmp/svd_u_2_3.tif)
v_geotiff_hdfs_paths = Array(hdfs:///user/pheno/svd/spark/BloomFinalLowPRSOSTLowPR3/v_tiffs/svd_v_0_3.tif, hdfs:///user/pheno/svd/spark/BloomFinalLowPRSOSTLowPR3/v_tiffs/svd_v_1_3.tif, hdfs:///user/pheno/svd/spark/BloomFinalLowPRSOSTLowPR3/v_tiffs/svd_v_2_3.tif)
v_geotiff_tmp_paths = Array(/tmp/svd_v_0_3.tif, /tmp/svd_v_1_3.tif, /tmp/svd_v_2_3.tif)
cmd = hadoop dfs -mkdir hdf...


hadoop dfs -mkdir hdfs:///user/pheno/svd/spark/BloomFinalLowPRSOSTLowPR3/v_tiffs/

## SVD

In [18]:
val svd: SingularValueDecomposition[IndexedRowMatrix, Matrix] = matrix_mul.computeSVD(n_components.toInt, computeU = true)

[Stage 157:=====================================================> (29 + 1) / 30]

svd = 


-0.0031281087007062826  3.59889857364454E-4     -2.57503458...


SingularValueDecomposition(org.apache.spark.mllib.linalg.distributed.IndexedRowMatrix@1d2a4a41,[1.2058745930351883E10,1.6180989320622066E7,7149599.582807035],-0.003094082471529021   2.751008759221239E-4    -3.495392832804974E-4   
-0.0030982550899576527  -5.077106383386543E-4   -9.506898530275529E-4   
-0.003103467512738654   1.2189548021424712E-4   -5.262205488864441E-6   
-0.003120955465414803   -1.7211863725889273E-4  -5.366885485462618E-4   
-0.003120955465414802   -1.721186372601296E-4   -5.36688548552774E-4    
-0.003144528624468846   -2.5694479469636503E-4  -3.506686153675054E-4   
-0.0031281087007062826  3.59889857364454E-4     -2.575034589537284E-4   
-0.003097208015263957   4.970262360332279E-4    4.856529696598507E-4    
-0.0031176622400108313  -6.696045355606668E-5   1.2973119728400826E-4   
-0.003117662240010832   -6.696045355532517E-5   1.2973119728789447E-4   
-0.003103452580306946   5.460274068996505E-5    2.8749117769129046E-4   
-0.003077470745398415   3.0288884711260

In [19]:
val U: IndexedRowMatrix = svd.U // The U factor is a RowMatrix.
val s: Vector = svd.s // The singular values are stored in a local dense vector.
val V: Matrix = svd.V // The V factor is a local dense matrix.
val S = Matrices.diag(s)

U = org.apache.spark.mllib.linalg.distributed.IndexedRowMatrix@1d2a4a41
s = [1.2058745930351883E10,1.6180989320622066E7,7149599.582807035]
V = 


-0.003097208015263957   4.970...


-0.003094082471529021   2.751008759221239E-4    -3.495392832804974E-4   
-0.0030982550899576527  -5.077106383386543E-4   -9.506898530275529E-4   
-0.003103467512738654   1.2189548021424712E-4   -5.262205488864441E-6   
-0.003120955465414803   -1.7211863725889273E-4  -5.366885485462618E-4   
-0.003120955465414802   -1.721186372601296E-4   -5.36688548552774E-4    
-0.003144528624468846   -2.5694479469636503E-4  -3.506686153675054E-4   
-0.0031281087007062826  3.59889857364454E-4     -2.575034589537284E-4   
-0.003097208015263957   4.970262360332279E-4    4.856529696598507E-4    
-0.0031176622400108313  -6.696045355606668E-5   1.2973119728400826E-4   
-0.003117662240010832   -6.696045355532517E-5   1.2973119728789447E-4   
-0.003103452580306946   5.460274068996505E-5    2.8749117769129046E-4   
-0.003077470745398415   3.0288884711260653E-4   -2.0180083587448402E-4  
-0.003079888834805732   -3.2934449470587584E-4  1.030135767536508E-4    
-0.0030560873832695826  -3.8299400269064556E-4  8.6

# Save Matrices

## R

In [30]:
val R_path = out_path + "U_matrix"
val U_RDD = U.rows.sortBy(_.index).map(_.vector.toArray)
U_RDD.cache()
U_RDD.saveAsObjectFile(R_path)

[Stage 278:===================================================>   (28 + 2) / 30]

R_path = hdfs:///user/pheno/svd/spark/BloomFinalLowPRSOSTLowPR3/U_matrix


hdfs:///user/pheno/svd/spark/BloomFinalLowPRSOSTLowPR3/U_matrix

# V

In [21]:
val V_path = out_path + "V_matrix"
val V_RDD = sc.parallelize(V.rowIter.toVector)
V_RDD.saveAsObjectFile(V_path)

[Stage 159:===================================================>   (28 + 2) / 30]

V_path = hdfs:///user/pheno/svd/spark/BloomFinalLowPRSOSTLowPR3/V_matrix
V_RDD = ParallelCollectionRDD[286] at parallelize at <console>:67


ParallelCollectionRDD[286] at parallelize at <console>:67

# S

In [22]:
val S_path = out_path + "S_matrix"
val S_RDD = sc.parallelize(S.rowIter.toVector)
S_RDD.saveAsObjectFile(S_path)

[Stage 160:===================================================>   (28 + 2) / 30]

S_path = hdfs:///user/pheno/svd/spark/BloomFinalLowPRSOSTLowPR3/S_matrix
S_RDD = ParallelCollectionRDD[289] at parallelize at <console>:68


ParallelCollectionRDD[289] at parallelize at <console>:68

# Save results

### R

In [23]:
val u_path = out_path + "U" + matrix_mode_str +".csv"
if (fs.exists(new org.apache.hadoop.fs.Path(u_path))) {
  cmd = "hadoop dfs -rm -r " + u_path
  Process(cmd)!
}
U_RDD.map(m => m.mkString(",")).saveAsTextFile(u_path)

[Stage 167:=================================================>     (27 + 3) / 30]

u_path = hdfs:///user/pheno/svd/spark/BloomFinalLowPRSOSTLowPR3/U.csv
U_RDD = MapPartitionsRDD[297] at map at <console>:78


MapPartitionsRDD[297] at map at <console>:78

### V

In [26]:
val v_path = out_path + "V" + matrix_mode_str +".csv"
if (fs.exists(new org.apache.hadoop.fs.Path(v_path))) {
  cmd = "hadoop dfs -rm -r " + v_path
  Process(cmd)!
}
sc.parallelize(V.rowIter.toVector.map(m => m.toArray.mkString(",")),4).saveAsTextFile(v_path)

[Stage 172:==========================================>              (3 + 1) / 4]

v_path = hdfs:///user/pheno/svd/spark/BloomFinalLowPRSOSTLowPR3/V.csv


hdfs:///user/pheno/svd/spark/BloomFinalLowPRSOSTLowPR3/V.csv

### S

In [27]:
val s_path = out_path + "S" + matrix_mode_str +".csv"
if (fs.exists(new org.apache.hadoop.fs.Path(s_path))) {
  cmd = "hadoop dfs -rm -r " + s_path
  Process(cmd)!
}
sc.parallelize(S.rowIter.toVector.map(m => m.toArray.mkString(",")),1).saveAsTextFile(s_path)

s_path = hdfs:///user/pheno/svd/spark/BloomFinalLowPRSOSTLowPR3/S.csv


hdfs:///user/pheno/svd/spark/BloomFinalLowPRSOSTLowPR3/S.csv

# Save GeoTiffs

## Create GeoTiffs for U (dimension is M(A) x n_components)

In [28]:
t0 = System.nanoTime()
val mod_grid0_index_I = model_grid0_index.zipWithIndex().map{ case (v,i) => (i,v)}
mod_grid0_index_I.cache()
model_grid0.cache()
cfor(0)(_ < n_components, _ + 1) { k =>
  //Merge two RDDs, one containing the clusters_ID indices and the other one the indices of a Tile's grid cells
  val kB = sc.broadcast(k)
  //val U_k_RDD = U.rows.map(_.toArray.zipWithIndex.filter(_._2 == kB.value).sortBy(_._2).map{ case (v,i) => v}).flatMap(m => m)
  val U_k_RDD = U_RDD.map(_(kB.value))
  val cluster_cell_pos = ((U_k_RDD.zipWithIndex().map{ case (v,i) => (i,v)}).join(mod_grid0_index_I)).map{ case (k,(v,i)) => (v,i)}
  cluster_cell_pos.cache()
    
  //Associate a Cluster_IDs to respective Grid_cell
  val grid_clusters :RDD[ (Long, (Double, Option[Double]))] = model_grid0.leftOuterJoin(cluster_cell_pos.map{ case (c,i) => (i, c)})

  //Convert all None to NaN
  val grid_clusters_res = grid_clusters.sortByKey(true).map{case (k, (v, c)) => if (c == None) (k, Double.NaN) else (k, c.get)}

  //Define a Tile
  val cluster_cells :Array[Double] = grid_clusters_res.values.collect()
  val cluster_cellsD = DoubleArrayTile(cluster_cells, num_cols_rows._1, num_cols_rows._2)
  val geoTif = new SinglebandGeoTiff(cluster_cellsD, projected_extent.extent, projected_extent.crs, Tags.empty, GeoTiffOptions(compression.DeflateCompression))
  cluster_cell_pos.unpersist()
    
  //Save to /tmp/
  GeoTiffWriter.write(geoTif, u_geotiff_tmp_paths(k))

  //Upload to HDFS
  var cmd = "hadoop dfs -copyFromLocal -f " + u_geotiff_tmp_paths(k) + " " + u_geotiff_hdfs_paths(k)
  Process(cmd)!

  //Remove from /tmp/
  cmd = "rm -fr " + u_geotiff_tmp_paths(k)
  Process(cmd)!
}
U_RDD.unpersist()
model_grid0.unpersist()
mod_grid0_index_I.unpersist()
t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

[Stage 195:===============================================>       (26 + 2) / 30]DEPRECATED: Use of this script to execute hdfs command is deprecated.
Instead use the hdfs command for it.

[Stage 216:=====================================================> (29 + 1) / 30]DEPRECATED: Use of this script to execute hdfs command is deprecated.
Instead use the hdfs command for it.

[Stage 237:=================================================>     (27 + 2) / 30]DEPRECATED: Use of this script to execute hdfs command is deprecated.
Instead use the hdfs command for it.

Elapsed time: 67667513733ns


t0 = 15231585110629
mod_grid0_index_I = MapPartitionsRDD[307] at map at <console>:80
t1 = 15299252624362


15299252624362

## Create GeoTiffs for V (dimension is M(B) x n_components)

In [52]:
t0 = System.nanoTime()
val sat_grid0_index_I = satellite_grid0_index.zipWithIndex().map{ case (v,i) => (i,v)}
val iter = V.colIter
var k :Int = 0
while (iter.hasNext) {
  //Merge two RDDs, one containing the clusters_ID indices and the other one the indices of a Tile's grid cells
  val V_k_RDD = sc.parallelize(iter.next().toArray.zipWithIndex().map{ case (v,i) => (i.toLong,v)})
  val cluster_cell_pos = (V_k_RDD.join(sat_grid0_index_I)).map{ case (k,(v,i)) => (v,i)}

  //Associate a Cluster_IDs to respective Grid_cell
  val grid_clusters :RDD[ (Long, (Double, Option[Double]))] = satellite_grid0.leftOuterJoin(cluster_cell_pos.map{ case (c,i) => (i, c)})

  //Convert all None to NaN
  val grid_clusters_res = grid_clusters.sortByKey(true).map{case (k, (v, c)) => if (c == None) (k, Double.NaN) else (k, c.get)}

  //Define a Tile
  val cluster_cells :Array[Double] = grid_clusters_res.values.collect()
  val cluster_cellsD = DoubleArrayTile(cluster_cells, num_cols_rows._1, num_cols_rows._2)
  val geoTif = new SinglebandGeoTiff(cluster_cellsD, projected_extent.extent, projected_extent.crs, Tags.empty, GeoTiffOptions(compression.DeflateCompression))

  //Save to /tmp/
  GeoTiffWriter.write(geoTif, v_geotiff_tmp_paths(k))

  //Upload to HDFS
  var cmd = "hadoop dfs -copyFromLocal -f " + v_geotiff_tmp_paths(k) + " " + v_geotiff_hdfs_paths(k)
  Process(cmd)!

  //Remove from /tmp/
  cmd = "rm -fr " + v_geotiff_tmp_paths(k)
  Process(cmd)!
    
  k += 1
}
t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

[Stage 341:===================================================>   (28 + 2) / 30]DEPRECATED: Use of this script to execute hdfs command is deprecated.
Instead use the hdfs command for it.

[Stage 353:=================================================>     (27 + 3) / 30]DEPRECATED: Use of this script to execute hdfs command is deprecated.
Instead use the hdfs command for it.

[Stage 365:=============================================>         (25 + 2) / 30]DEPRECATED: Use of this script to execute hdfs command is deprecated.
Instead use the hdfs command for it.

Elapsed time: 53006246028ns


t0 = 16674678706701
sat_grid0_index_I = MapPartitionsRDD[437] at map at <console>:90
iter = empty iterator
k = 3
t1 = 16727684952729


16727684952729